In [13]:
%pip install pyradiomics dicom_numpy pydicom plotly matplotlib scikit-image simpleITK pynrrd dicom2nifti NiBabel NiLearn openpyxl pydicom-seg itkwidgets

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider
from matplotlib import patches, text, patheffects

import dicom_numpy
import pydicom as dicom

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import SimpleITK as sitk
import os

from radiomics import featureextractor

from tqdm import tqdm
import itertools
from functools import wraps

In [15]:
annotation_boxes = pd.read_excel("../../Simple Path Dataset/Annotation_Boxes.xlsx").set_index("Patient ID")



DATASET_PATH = '../../Processed NIFTI Dataset/'

In [16]:
annotation_boxes

,Start Row,End Row,Start Column,End Column,Start Slice,End Slice
Patient ID,,,,,,
Breast_MRI_001,234,271,308,341,89,112
Breast_MRI_002,251,294,108,136,59,72
Breast_MRI_003,351,412,82,139,96,108
Breast_MRI_004,262,280,193,204,86,95
Breast_MRI_005,188,213,138,178,76,122
...,...,...,...,...,...,...
Breast_MRI_918,345,395,338,395,62,85
Breast_MRI_919,285,312,369,397,98,109
Breast_MRI_920,172,193,337,355,87,101


In [17]:
annotation_boxes.loc['Breast_MRI_001']

Start Row       234
End Row         271
Start Column    308
End Column      341
Start Slice      89
End Slice       112
Name: Breast_MRI_001, dtype: int64

In [18]:
# filenames = ['pre.nii.gz', 'post_1.nii.gz']

In [19]:
def relu(x):
    return np.maximum(x, 0)

In [20]:
annotations_nifti = pd.DataFrame()

In [21]:
# x2 = x0+(x-x0)*cos(theta)+(y-y0)*sin(theta)
# y2 = y0-(x-x0)*sin(theta)+(y-y0)*cos(theta)

# x2 = x0 + (x-x0)* 0 + y - y0 

In [27]:
items = os.listdir(DATASET_PATH)

for patient in tqdm(items):
# for item in tqdm(os.listdir(DATASET_PATH)):
    patient_path = os.path.join(DATASET_PATH, patient)

    precontrast = nib.load(patient_path + '/pre.img.gz')
    postcontrast = nib.load(patient_path + '/post_1.img.gz')

    height, width, depth = precontrast.shape
    x = annotation_boxes.loc[patient]
    row1 = x['Start Row']
    row2 = x['End Row']       
       
    col1 = x['Start Column']
    col2 = x['End Column']
    
    slice1 = x['Start Slice']
    slice2 = x['End Slice']

    # slice1 = depth - x['End Slice']
    # slice2 = depth - x['Start Slice']
    
    annotate = pd.DataFrame([[patient, row1, row2, col1, col2, slice1, slice2]], 
                            columns = ['Patient_ID', 'Start Row', 'End Row', 'Start Column', 'End Column', 'Start Slice', 'End Slice']
                            )
    annotations_nifti = pd.concat([annotations_nifti, annotate], axis = 0)


100%|██████████| 922/922 [00:03<00:00, 262.33it/s]


## THIS JUST SETS NEW ANNOTATIONS TO OLD ANNOTATIONS

In [28]:
annotations_nifti.to_csv("segmentation_annotations_NIFTI.csv", index = False)
annotations_nifti.to_csv("../Data/segmentation_annotations_NIFTI.csv", index = False)


# annotation_boxes.reset_index().to_csv("segmentation_annotations_NIFTI.csv", index = False)
# annotation_boxes.reset_index().to_csv("../Data/segmentation_annotations_NIFTI.csv", index = False)